In [1]:
import client
import os
import time

conn = client.IPPClient(os.path.expanduser("~/.ipython/profile_default/security/ipcontroller-client.json"))
client = conn.get_client()

In [128]:
DEBUG=True

def debug_print(*args, **kwargs):
    if DEBUG:
        print('DEBUG: ', *args, **kwargs)
        
debug_print("{0} - {1}".format(4,5))
debug_print(4,5)

DEBUG:  4 - 5
DEBUG:  4 5


In [2]:
class APP (object):
    
    def __init__ (self, executable, inputs=[], outputs=[], env={}, walltime=60, exec_type="bash"):
        self.executable = executable
        self.inputs     = inputs
        self.outputs    = outputs
        self.exec_type  = exec_type
        self.status     = 'created'
        self.stdout     = None
        self.stderr     = None
            
    def __call__(self, block=True):
        
        import time
        import subprocess
        start_t = time.time()
        if self.exec_type != "bash":
            raise NotImplemented        

        self.stdout = "exec_wait.out.txt"
        self.stderr = "exec_wait.err.txt"
        std_out = open(self.stdout, 'w')
        std_err = open(self.stderr, 'w')
        start_time = time.time()    
        try :
            proc = subprocess.Popen(self.executable, stdout=std_out, stderr=std_err, shell=True)
            proc.wait()
        except Exception as e:
            print("Caught exception : {0}".format(e))
            self.error = e
            self.status = 'failed'
            return -1
        
        self.exec_duration = time.time() - start_t
        print("RunCommand Completed {0}".format(self.executable))
        return self.exec_duration
        

In [157]:
dview=client[:]
@dview.remote(block=False)
def execute(desc, block=False):        
    import time
    import os
    import subprocess
    start_t = time.time()
    if desc["exec_type"] != "bash":
        raise NotImplemented        

    #desc["stdout"] = "exec_wait.out.txt"
    #desc["stderr"] = "exec_wait.err.txt"
    std_out = open(desc["stdout"], 'w')
    std_err = open(desc["stderr"], 'w')
    start_time = time.time()    
    try :
        proc = subprocess.Popen(desc["executable"], stdout=std_out, stderr=std_err, shell=True)
        proc.wait()
    except Exception as e:
        print("Caught exception : {0}".format(e))
        desc["error"] = e
        desc["status"] = 'failed'
        return -1
        
    desc["exec_duration"] = time.time() - start_t
    desc["pid"] = os.getpid()
    print("RunCommand Completed {0}".format(desc["executable"]))
    return desc

In [156]:
import time
app_def = { "exec_type"   : "bash",
            "stdout"      : None,
            "stderr"      : None,
            "executable"  : "echo {inputs[0]} {inputs{0}} &> {outputs[0]}",
            "inputs"      : ["hello", "world"],            
            "outputs"     : ["output.txt"],      
           }

start = time.time()
res = [(i, execute(app_def)) for i in range(0, 10000)]
print("total_dur : ", time.time()-start)

total_dur :  33.27006149291992


In [38]:
print(res)

[{'stdout': 'exec_wait.out.txt', 'outputs': ['output.txt'], 'executable': 'echo {inputs[0]} {inputs{0}} &> {outputs[0]}', 'exec_type': 'bash', 'stderr': 'exec_wait.err.txt', 'pid': 1655, 'inputs': ['hello', 'world'], 'exec_duration': 0.003741741180419922}]


In [133]:
import sys

class APP(object):
    
    def __init__ (self, f, desc):
        self.func = f
        self.desc = desc        
        
    def __call__(self, *args, **kwargs):
        debug_print("Tracer")
        def tracer(frame, event, arg):
            if event=='return':
                self._locals = frame.f_locals.copy()
        
        # Activate tracer
        sys.setprofile(tracer)
        try:
            # trace the function call
            res = self.func(*args, **kwargs)
        finally:
            # disable tracer and replace with old one
            sys.setprofile(None)

        cmd_line = self._locals['cmd_line']
        debug_print('Command line : ', cmd_line)
        debug_print('kwargs       : ', kwargs)
        formatted = cmd_line.format(**kwargs)
        
        self.desc.update(kwargs)
        self.desc.update({'executable' : formatted})
        
        return execute(self.desc)        
       
def App(apptype):
    # App Decorator
    debug_print ("Apptype", apptype)
    app_def = { "exec_type" : apptype, 
                "inputs" : [],
                "outputs" : [],
                "env" : {} }
        
    def Exec(f):
        debug_print("Decorator Exec : ", f)                
        #print("In the inner function with arg f : {0}".format(str(f.__code__)) )        
        #print("Def  : {0}".format(app_def))        
        return APP(f, app_def)
    
    return Exec

In [150]:
###############################################################################################
@App('bash')
def echo(inputs=[], stderr='std.err', stdout='std.out'):        
    cmd_line = 'echo {inputs[0]} {inputs[1]}'    
###############################################################################################
    
result = echo(inputs=["Hello", "World!"], stdout='std.out', stderr='std.err')
print(result)
print(result.get())

<AsyncResult: execute>
[{'stdout': 'exec_wait.out.txt', 'outputs': [], 'executable': 'echo Hello World!', 'env': {}, 'exec_type': 'bash', 'stderr': 'exec_wait.err.txt', 'pid': 1655, 'inputs': ['Hello', 'World!'], 'exec_duration': 0.004425048828125}]


In [145]:
DEBUG=False
###############################################################################################
@App('bash')
def sleep(sleeptime=0, inputs=[], stderr='std.err', stdout='std.out'):        
    cmd_line = 'sleep {sleeptime}'    
###############################################################################################    
result = sleep(sleeptime=1)

print("Got the future from the sleep app :", result)


def poll(asyncresult, duration=1, maxwait=10, verbose=True):
    for i in range(duration*maxwait):
        if asyncresult.ready():
            if verbose:
                print("Done at {0}".format(duration*i))
            return True
        else:
            if verbose:
                print("Sleeping at {0}".format(duration*i))
            time.sleep(duration)
            
poll(result)
print(result.get())

Got the future from the sleep app : <AsyncResult: execute>
Sleeping at 0
Sleeping at 1
Done at 2
[{'stdout': 'exec_wait.out.txt', 'outputs': [], 'executable': 'sleep 1', 'env': {}, 'stderr': 'exec_wait.err.txt', 'exec_type': 'bash', 'sleeptime': 1, 'pid': 1655, 'inputs': [], 'exec_duration': 1.0055561065673828}]


In [149]:
results = []
for i in range(0,5):
    #print(i)
    x= sleep(sleeptime=i)
    results.append(x)
    

    
print(results)
time.sleep(3)
print(results)
print( [i.get() for i in results if i.ready()] )


0
1
2
3
4
[<AsyncResult: execute>, <AsyncResult: execute>, <AsyncResult: execute>, <AsyncResult: execute>, <AsyncResult: execute>]
[<AsyncResult: execute:finished>, <AsyncResult: execute>, <AsyncResult: execute>, <AsyncResult: execute>, <AsyncResult: execute>]
[[{'stdout': 'exec_wait.out.txt', 'outputs': [], 'executable': 'sleep 0', 'env': {}, 'stderr': 'exec_wait.err.txt', 'exec_type': 'bash', 'sleeptime': 0, 'pid': 1655, 'inputs': [], 'exec_duration': 0.0063152313232421875}]]


In [ ]:


###############################################################################################
@App('bash')
def cat(inputs=[''], stderr='std.err', stdout='std.out'):        
    cmd_line = 'sleep {sleeptime}'
###############################################################################################



In [151]:
cmd_line = 'echo "{inputs[0]} {inputs[1]}"'
kwargs = {'inputs' :["Hello", "World"],
          'outputs' : []}
fo = cmd_line.format(**kwargs)
print(fo)

echo "Hello World"


In [ ]:
Show out of order eval of remote functions
Put a regular function behind the future
